In [132]:
import pandas as pd
from tqdm import tqdm
import json

In [102]:
dfs = []

tmp = pd.read_json('../query/Inferred_Multiple_Wish_Query_Meta_Train.json', lines=True)
tmp['split'] = 'infer_train'
dfs.append(tmp)

tmp = pd.read_json('../query/Inferred_Multiple_Wish_Query_Meta_Val.json', lines=True)
tmp['split'] = 'infer_val'
dfs.append(tmp)

tmp = pd.read_json('../query/Inferred_Multiple_Wish_Query_Meta_Test.json', lines=True)
tmp['split'] = 'infer_test'
dfs.append(tmp)

tmp = pd.read_json('processed2/Human_Labelled_Query_Classification_Train.json', lines=True)
tmp['split'] = 'label_train'
dfs.append(tmp)

tmp = pd.read_json('processed2/Human_Labelled_Query_Classification_Val.json', lines=True)
tmp['split'] = 'label_val'
dfs.append(tmp)

tmp = pd.read_json('processed2/Human_Labelled_Query_Classification_Test.json', lines=True)
tmp['split'] = 'label_test'
dfs.append(tmp)

df = pd.concat(dfs)
del dfs

In [103]:
df_group = df.groupby('query').agg({'category': lambda x: [i for i in x], 'split': lambda x: [i for i in x]})

In [104]:
len(df_group)

2846418

In [105]:
df_group = df_group.reset_index()

In [106]:
queries_to_rm_from_infer = set(df_group[df_group.split.apply(lambda x: len(set(x)) > 1 and ('label_test' in x or 'label_val' in x))]['query'])

In [107]:
recs = []
for i in tqdm(df_group.to_dict('records')):
    cats = [] 
    splits = []
    if i['query'] in queries_to_rm_from_infer:
        for cat, split in zip(i['category'], i['split']):
            if 'infer' not in split:
                cats.append(cat)
                splits.append(split)
        i['category'] = cats 
        i['split'] = splits 
    recs.append(i)
            

100%|██████████| 2846418/2846418 [00:02<00:00, 997129.68it/s] 


In [108]:
df_group = pd.DataFrame(recs)

In [109]:
def remove_prefix_strings(strings_in):
    res = set(strings_in)
    strings = sorted(res, key=len)
    for i in range(len(strings)):
        for j in range(i+1, len(strings)):
            if strings[j].startswith(strings[i]):
                res.remove(strings[i])
                break
    return list(res)

In [110]:
remove_prefix_strings(['beauty & health > sexual wellness > safer sex > lubricants',
   'beauty & health > sexual wellness > sex toys > afrodisiac',
   'beauty & health',
   'beauty & health > sexual wellness > safer sex > lubricants'])

['beauty & health > sexual wellness > sex toys > afrodisiac',
 'beauty & health > sexual wellness > safer sex > lubricants']

In [111]:
df_group['category_dedup'] = df_group['category'].apply(remove_prefix_strings)

In [112]:
df_group[df_group.category_dedup.apply(lambda x: 'unknown' in x and len(x) > 1)].sample(5).to_dict('records')

[{'query': 'purchases',
  'category': ["women's clothing > tops > tees",
   'automobiles & motorcycles > motorcycle accessories & parts > protective gear > shirts & tops',
   "women's clothing",
   'automobiles & motorcycles > motorcycle accessories & parts',
   'unknown'],
  'split': ['infer_train',
   'infer_train',
   'infer_train',
   'infer_train',
   'label_train'],
  'category_dedup': ["women's clothing > tops > tees",
   'unknown',
   'automobiles & motorcycles > motorcycle accessories & parts > protective gear > shirts & tops']},
 {'query': 'blue',
  'category': ['toys & hobbies > stuffed animals & plush toys > movies & tv',
   'jewelry & accessories > necklaces & pendants > pendants',
   'jewelry & accessories > fine jewelry > earrings',
   'unknown'],
  'split': ['infer_train', 'infer_train', 'infer_train', 'label_train'],
  'category_dedup': ['unknown',
   'jewelry & accessories > necklaces & pendants > pendants',
   'jewelry & accessories > fine jewelry > earrings',
   'to

In [121]:
df_group.loc[df_group.category_dedup.apply(lambda x: 'unknown' in x), 'category_dedup'] = df_group.loc[
    df_group.category_dedup.apply(lambda x: 'unknown' in x), 'category_dedup'].apply(lambda x: ["unknown"])

In [131]:
df_group[df_group.category_dedup.apply(lambda x: 'unknown' in x)].sample(5).to_dict('records')

[{'query': 'newest',
  'category': ['unknown'],
  'split': ['label_train'],
  'category_dedup': ['unknown']},
 {'query': 'brindes gratis',
  'category': ['unknown'],
  'split': ['label_train'],
  'category_dedup': ['unknown']},
 {'query': 'samen pflanzen',
  'category': ['unknown'],
  'split': ['label_test'],
  'category_dedup': ['unknown']},
 {'query': 'resortera',
  'category': ['toys & hobbies > outdoor fun & sports > toy sports',
   'toys & hobbies > outdoor fun & sports',
   'toys & hobbies',
   'unknown'],
  'split': ['infer_train', 'infer_train', 'infer_train', 'label_train'],
  'category_dedup': ['unknown']},
 {'query': 'return policy',
  'category': ['home & garden > festive & party supplies > event & party > party favors',
   'home & garden > festive & party supplies > event & party',
   'home & garden',
   'unknown'],
  'split': ['infer_train', 'infer_train', 'infer_train', 'label_train'],
  'category_dedup': ['unknown']}]

In [124]:
df_group_val = df_group[df_group.split.apply(lambda x: "label_val" in x)]
df_group_test = df_group[df_group.split.apply(lambda x: "label_test" in x)]
df_group_train_labelonly = df_group[df_group.split.apply(lambda x: "label_train" in x and "infer_train" not in x)]
df_group_train_inferonly = df_group[df_group.split.apply(lambda x: "infer_train" in x and "label_train" not in x)]
df_group_train_label_and_infer = df_group[df_group.split.apply(lambda x: "infer_train" in x and "label_train" in x)]
len(df_group_val), len(df_group_test), len(df_group_train_labelonly), len(df_group_train_inferonly), len(df_group_train_label_and_infer), len(df_group)

(5000, 4300, 17126, 2498981, 43294, 2846418)

In [125]:
5000 + 4300 + 17126 + 2498981 + 43294

2568701

In [126]:
df_group.category_dedup.apply(len).mean(), df_group.category.apply(len).mean()

(1.2041028408336372, 3.0389074970717584)

In [127]:
assert df_group_test.split.apply(lambda x: len(set(x)) == 1).all()
assert df_group_val.split.apply(lambda x: len(set(x)) == 1).all()

In [128]:
test_dup_dfs = [df_group_test, df_group_val, df_group_train_labelonly, df_group_train_inferonly, df_group_train_label_and_infer]
for i in range(len(test_dup_dfs)):
    for j in range(i+1, len(test_dup_dfs)):
        assert len(set(test_dup_dfs[i]['query']).intersection(set(test_dup_dfs[j]['query']))) == 0

In [133]:
with open("processed3/Human_Labelled_Query_Classification_Test_DedupOverlap.json", 'w') as f:
    for i in df_group_test.to_dict('records'):
        for j in i['category_dedup']:
            f.write(json.dumps({"query": i["query"], "category": j}) + "\n")
with open("processed3/Human_Labelled_Query_Classification_Val_DedupOverlap.json", 'w') as f:
    for i in df_group_val.to_dict('records'):
        for j in i['category_dedup']:
            f.write(json.dumps({"query": i["query"], "category": j}) + "\n")
with open("processed3/OnlyHuman_Labelled_Query_Classification_Train_DedupOverlap.json", 'w') as f:
    for i in df_group_train_labelonly.to_dict('records'):
        for j in i['category_dedup']:
            f.write(json.dumps({"query": i["query"], "category": j}) + "\n")
with open("processed3/OnlyInferred_Query_Classification_Train_DedupOverlap.json", 'w') as f:
    for i in df_group_train_inferonly.to_dict('records'):
        for j in i['category_dedup']:
            f.write(json.dumps({"query": i["query"], "category": j}) + "\n")
with open("processed3/Mixed_Human_Inferred_Query_Classification_Train_DedupOverlap.json", 'w') as f:
    for i in df_group_train_label_and_infer.to_dict('records'):
        for j in i['category_dedup']:
            f.write(json.dumps({"query": i["query"], "category": j}) + "\n")